---
## Bu .ipynb dosyası 2.adım işlemidir.Burda yapılan işlemler adım-1 in üstüne koyularak yapılacaktır.
---


### Önce Kod :: :: Adım 1'in üstüne neler eklediğimize bu klasörden ulaşabilirsiniz. :

**"Attention Mekanizmaları\Cordinat Attention (3.1)\Adım-2\Bu_adımda_neler_ekledik.ipynb"**

-----

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0
# Adım 1 de olan adımın aynısı uygulandı.Relu6 fonksiyonundan HSwish e uyarlama yaptık.Daha ucuz ve daha yumuşak davranır.
# Matematik denklemi :: :: :: x * relu(x+3) / 6

def make_norm(norm: str, ch: int):
    if norm == "bn":
        return nn.BatchNorm2d(ch)

    if norm == "gn":
        g = min(32, ch)  # GN'de num_groups <= num_channels olmalı. Bu yüzden en fazla ch, tercihen 32 ile başlıyoruz.

        # GN şartı: ch % g == 0 olmalı (kanallar gruplara eşit bölünsün).
        # Uymuyorsa g'yi ikiye bölerek 32→16→8→4→2 şeklinde düşürüp uygun bir bölen arıyoruz.
        while ch % g != 0 and g > 2:
            g //= 2  # g'yi yarıya indir (tam sayı bölmesi)

        # Eğer hâlâ bölünmüyorsa (özellikle tek kanal sayılarında), son çare:
        # ch çiftse 2 gruba böl, tekse 1 grup (tüm kanallar tek grupta) kullan.
        if ch % g != 0:
            g = 2 if (ch % 2 == 0) else 1

        return nn.GroupNorm(g, ch)

    if norm == "none":
        return nn.Identity()

    raise ValueError("norm must be 'bn', 'gn', or 'none'")


class CoordinateAttPlus(nn.Module):
    def __init__(
        self,
        in_channels: int,
        reduction: int = 32,
        min_mid_channels: int = 8,
        act: str = "hswish",
        init_alpha: float = 0.7,
        learnable_alpha: bool = True,
        beta: float = 0.35,
        dilation: int = 2,
        norm: str = "gn",
        use_spatial_gate: bool = False,
        spatial_gate_beta: float = 0.35,
    ):
        super().__init__()
        mid = max(min_mid_channels, in_channels // reduction) 
        ## Burası bizim bottleneck ifademiz.Aslında burda yapılan şey;
            # H+W birleşik şerit üzerinde ortak karar uzayı: C→mid sıkıştırma + mid→mid refine.”

        self.shared_bottleneck_proj = nn.Conv2d(in_channels, mid, 1, bias=False)
        self.shared_bottleneck_norm = make_norm(norm, mid)
        self.shared_bottleneck_refine = nn.Conv2d(mid, mid, 1, bias=False)
        self.shared_bottleneck_refine_norm = make_norm(norm, mid)
        ## Burası bottlenecek mimarisi.Şu an katman karşılığı
            # in_channels -> mid -> mid 
                # DİKKAT :: Burda henüz in_channels ile bir bağlantı yapmadık.Bu sonraki adım.O adımı aşağıda attention mekanizmasında göreceğiz

        if act.lower() == "hswish":
            self.act = HSwish()
        elif act.lower() == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act must be 'hswish' or 'relu'")
        
        self.h_local_dw = nn.Conv2d(in_channels, in_channels, (3, 1), padding=(1, 0), groups=in_channels, bias=False)
        ## H için yapılan depthwise conv işlemi.Yalnıca H olduğu için :
            # Kernel (3,1) :: Yani H kanalına dikkat çektiriyoruz.Diğer taraftan:
                # Padding ifadesinde ise yalnzıca H eksenine padding yapıyoruz.W ekseni görüldüğü gibi 0.

        self.w_local_dw = nn.Conv2d(in_channels, in_channels, (1, 3), padding=(0, 1), groups=in_channels, bias=False)
        ## W için yapılan depthwise conv işlemidir.Yalnızca W ye uygulanır :
            # Kernel (1,3) olduğundan H eksenine değil W eksenine odaklanır.
                # Padding de aynı şekilde.Yalnızca W eksenine yapılır.


        d = int(dilation)
        self.h_dilated_dw = nn.Conv2d(in_channels,in_channels,(3, 1),padding=(d, 0),dilation=(d, 1),groups=in_channels,bias=False,)
        # H için yapılan dilated convdur.Yapı şudur:
            # Yine 3,1 kernel_size kullanılıyor.Diğer Tarafdan pad değeri dilation ile aynı.
                # Dilation da ise H değerine d veriyoruz.W boyutuna ise 1.Yani amacımız sadece H'a uygulamak

        
        self.w_dilated_dw = nn.Conv2d(in_channels,in_channels,(1, 3),padding=(0, d),dilation=(1, d),groups=in_channels,bias=False,)
        # Yukarıda olduğu gibi.W için yapılan işlemdir.

        self.h_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        # H kanalından gelen dilated ve depthwise işlemini burda 1x1 e sokuyoruz.
            # Amaç dilated ve depthwise dan gelen kanalları 1x1 conv'a koymak.
                # Depthwise çıktılarından sonra channel mixing (kanallar arası etkileşim) eklemek için 1×1.

        self.w_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        # W kanalından gelen bilgilerin de aynısı burda işlenir.

        self.h_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)
        # H ekseninine attention ın asıl uygulandığı katmandır.Yani aslında dikkat kısmını bütün channels'e gönderiyoruz
            # Yani örnekle mid -> channels === 8 -> 256 
                # Aslında karar mekanizmasına şunu dedik. 
                    # Bu head’ler mid → C projeksiyonu yapar.Sonra hardsigmoid ile maskeyi üretir.

        self.w_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)
        # W eksenindeki işlemi de burda yapıyoruz.

        self.beta = float(beta)
        # Attention ın 2.kapısıdır.Yine attention kontrol edilir.Fakat eksen olarak değil bütün attention kontrol edilir.
        # beta, final scale_hw’yi 1’e yaklaştırarak agresifliği kısar.Yani “global gain / damping”
        
        eps = 1e-6
        a0 = float(init_alpha)
            # --- alpha başlangıcı (logit uzayı) ---
        # Amaç: alpha değerini (0,1) aralığında tutmak ama öğrenmeyi serbest bırakmak.
        # Bunun için alpha'yı doğrudan öğrenmek yerine "alpha_raw" (logit) parametresini öğreniyoruz:
        #   alpha = sigmoid(alpha_raw)  -> (0,1)
        # init_alpha başlangıç gücüdür (ör. 0.7). 0 veya 1 olursa logit patlar, o yüzden eps ile clamp ediyoruz.
        #   a0 ∈ (eps, 1-eps)
        # raw0 = log(a0 / (1-a0))  -> sigmoid(raw0) = a0
        # Yani alpha_raw'ı raw0 ile başlatınca, ileri geçişte alpha ilk başta tam init_alpha olur.
        a0 = min(max(a0, eps), 1.0 - eps)
        t = torch.tensor(a0)
        raw0 = torch.log(t / (1.0 - t))

        # learnable_alpha=True ise alpha_raw öğrenilir (Parameter).
        # False ise sabit tutulur (buffer): eğitim boyunca değişmez, sadece kayıt/cihaz taşınır.
        if learnable_alpha:
            self.alpha_h_raw = nn.Parameter(raw0.clone())
            self.alpha_w_raw = nn.Parameter(raw0.clone())
        else:
            self.register_buffer("alpha_h_raw", raw0.clone())
            self.register_buffer("alpha_w_raw", raw0.clone())


        self.use_spatial_gate = bool(use_spatial_gate)
        self.spatial_gate_beta = float(spatial_gate_beta)
        if self.use_spatial_gate:
            self.spatial_gate_dw = nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels, bias=False)
            self.spatial_gate_pw = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        # --- Opsiyonel Spatial Gate (ek 2D lokal kapı) ---
        # use_spatial_gate=True ise, CA ile ölçeklenmiş çıktı üzerine bir de 2D (H,W) tabanlı gate ekleriz.
        # spatial_gate_dw: 3x3 depthwise -> lokal komşuluk bilgisi (kanal kanal)
        # spatial_gate_pw: 1x1 pointwise -> kanal karıştırma / yeniden ölçekleme
        # Bu gate sigmoid ile (0,1) aralığına sıkıştırılıp beta ile 1'e yaklaştırılarak "yumuşatılır".
        # Not: Hata/bug değildir; bazı senaryolarda fayda verir, bazı senaryolarda double-gating yüzünden over-suppression yapabilir.


        self._last_ah = None
        self._last_aw = None

    def forward(self, x):
        B, C, H, W = x.shape
        # Giriş shape'lerini al (B: batch, C: kanal, H: yükseklik, W: genişlik)

        H_profile = 0.5 * (x.mean(dim=3, keepdim=True) + x.amax(dim=3, keepdim=True))
        # W eksenini (dim=3) squeeze ederek (B,C,H,1) H-profili çıkar: mean + max karışımı (0.5 ile ortalama)

        W_profile = 0.5 * (x.mean(dim=2, keepdim=True) + x.amax(dim=2, keepdim=True))
        # H eksenini (dim=2) squeeze ederek (B,C,1,W) W-profili çıkar: mean + max karışımı (0.5 ile ortalama)

        H_multi_scale = self.h_channel_mixer(self.h_local_dw(H_profile) + self.h_dilated_dw(H_profile))
        # H-profili üzerinde local DW + dilated DW çıktısını topla (multi-scale), sonra 1x1 ile channel mixing yap -> (B,C,H,1)

        W_multi_scale = self.w_local_dw(W_profile) + self.w_dilated_dw(W_profile)
        # W-profili üzerinde local DW + dilated DW çıktısını topla (multi-scale) -> (B,C,1,W)

        W_multi_scale = self.w_channel_mixer(W_multi_scale)
        # Depthwise sonrası kanalları karıştırmak için 1x1 channel mixer uygula -> (B,C,1,W)

        W_multi_scale = W_multi_scale.permute(0, 1, 3, 2)
        # Concat için formatı değiştir: (B,C,1,W) -> (B,C,W,1)

        hw_concat = torch.cat([H_multi_scale, W_multi_scale], dim=2)
        # H ve W şeritlerini dim=2 boyunca yan yana ekle: (B,C,H,1)+(B,C,W,1) -> (B,C,H+W,1)

        shared_mid = self.act(self.shared_bottleneck_norm(self.shared_bottleneck_proj(hw_concat)))
        # Shared bottleneck 1. adım: kanal projeksiyonu C->mid + norm + aktivasyon -> (B,mid,H+W,1)

        shared_mid = self.act(self.shared_bottleneck_refine_norm(self.shared_bottleneck_refine(shared_mid)))
        # Shared bottleneck 2. adım: mid->mid refine + norm + aktivasyon -> (B,mid,H+W,1)

        mid_H, mid_W = torch.split(shared_mid, [H, W], dim=2)
        # H+W uzunluğunu tekrar ayır: ilk H parça (B,mid,H,1), ikinci W parça (B,mid,W,1)

        mid_W = mid_W.permute(0, 1, 3, 2)
        # W parçasını head için formata sok: (B,mid,W,1) -> (B,mid,1,W)

        attn_H = F.hardsigmoid(self.h_attention_head(mid_H), inplace=False)# Bu, maskenin kendisini üretmek için.
        # H head: mid->C projeksiyonu ve hardsigmoid ile (0,1) benzeri H maskesi üret -> (B,C,H,1)

        attn_W = F.hardsigmoid(self.w_attention_head(mid_W), inplace=False)# Bu, maskenin kendisini üretmek için.
        # W head: mid->C projeksiyonu ve hardsigmoid ile (0,1) benzeri W maskesi üret -> (B,C,1,W)

        self._last_ah = attn_H.detach()
        # Debug/istatistik için H maskesinin kopyasını graph'tan ayırıp sakla

        self._last_aw = attn_W.detach()
        # Debug/istatistik için W maskesinin kopyasını graph'tan ayırıp sakla


        alpha_h = torch.sigmoid(self.alpha_h_raw)# Bu ise maskeyi değil, maskenin gücünü ayarlayan karışım katsayısını üretmek için.
        # Bu alpha değerleri normalde -inf , inf aralığındadır.Biz sigmoid ile bunları 0,1 aralığına sıkıştırdık.                                                  
        alpha_w = torch.sigmoid(self.alpha_w_raw)# Bu ise maskeyi değil, maskenin gücünü ayarlayan karışım katsayısını üretmek için.                                                        

        scale_H = (1.0 - alpha_h) + alpha_h * attn_H  # Burda dierktr scale matematiğini kulanadık :: scale=(1−α)+α⋅attn                                                   
        scale_W = (1.0 - alpha_w) + alpha_w * attn_W                                                         

        scale_hw = scale_H * scale_W
         # Bir nokta ancak hem satırı hem sütunu önemliyse güçlü kalır.“İki koşul birlikte sağlansın” mantığı.      
         # (B,C,H,1) * (B,C,1,W) → (B,C,H,W)
                                                                       
        scale_hw = 1.0 + self.beta * (scale_hw - 1.0)                                                        

        out = x * scale_hw # her kanalın her pikseli, kendi ölçeğiyle çarpılıyor.                                                                               

        if self.use_spatial_gate:# Bu blok, CA’den sonra çıktıya ek bir 2D lokal kapı uygular; bazen faydalı, bazen fazla bastırıp performansı bozabilir.Eğer kullanılacaksa dikkat edilmeli.
            gate = torch.sigmoid(self.spatial_gate_pw(self.spatial_gate_dw(out)))                            
            gate = 1.0 + self.spatial_gate_beta * (gate - 1.0)                                              
            out = out * gate                                                                                 

        return out

    @torch.no_grad()
    def last_mask_stats(self):
        if self._last_ah is None or self._last_aw is None:
            return None
        ah = self._last_ah
        aw = self._last_aw
        return {
            "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
            "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
        }

----

### Kodun içerisindeki yorum satırlarını okuyunuz.Eğer anlam karmaşası yaşarsanız aşağıdaki ifadelerden daha net anlam çıkartabilirsiniz.Değişken isimleri değişebilir.Ama siz ana işleme odaklanınız....
----

# `make_norm()` Ne Yapıyor?

Bu fonksiyonun tek amacı: **Normalization katmanını seçmek**.

- `norm="bn"` → `BatchNorm2d(ch)`
- `norm="gn"` → `GroupNorm(g, ch)` (g otomatik seçilir)
- `norm="none"` → `Identity()` (hiç normalize etmez)



## BatchNorm (BN) vs GroupNorm (GN) — Temel Fark

### BatchNorm2d (BN)
**Nasıl çalışır?**
- Normalize işlemini **batch boyutunu da dahil ederek** yapar.
- Her kanal için istatistikleri (mean/var) **batch üzerinden** hesaplar.

**Artıları**
- Büyük batch ile genellikle iyi çalışır.
- Eğitimde hızlı ve stabil olabilir (uygun batch boyutunda).

**Eksileri**
- **Small batch** (özellikle detection / segmentation gibi) durumlarda istatistikler “gürültülü” olur → performans ve stabilite düşebilir.
- Train/test farkı: train’de batch istatistiği, test’te running mean/var kullanır.

**Ne zaman mantıklı?**
- Batch boyutu yeterince büyükse (ör. classification eğitimleri).


### GroupNorm (GN)
**Nasıl çalışır?**
- Normalize işlemini **batch’e bakmadan** yapar.
- Kanalları `g` adet gruba böler, her grup kendi içinde normalize edilir.
- Bu yüzden **batch boyutundan bağımsız** daha stabil davranır.

**Artıları**
- Small batch’te bile stabil.
- Detection gibi küçük batch kullanılan işlerde sık tercih edilir.

**Eksileri**
- Bazı durumlarda BN kadar hızlı “oturmayabilir” (tamamen probleme bağlı).

**Ne zaman mantıklı?**
- Batch küçükse veya batch değişkenliği fazlaysa (özellikle detection).


## GroupNorm’da Kritik Şart: `ch % g == 0`

GroupNorm şu şekilde tanımlanır:

- `GroupNorm(num_groups=g, num_channels=ch)`

Ama **kanallar gruplara eşit bölünebilmek zorunda** olduğu için:

> `ch % g == 0` olmalı.

Yani `g`, `ch`’nin tam böleni olmalı.


## `gn` Dalı: `g` Nasıl Seçiliyor?

Kod:

```python
g = min(32, ch)
while ch % g != 0 and g > 2:
    g //= 2
if ch % g != 0:
    g = 2 if (ch % 2 == 0) else 1
return nn.GroupNorm(g, ch)
```

### Mini Örnekler

* ch = 120

* g=32 → 120%32≠0 → g=16 → 120%16≠0 → g=8 → 120%8=0

* Sonuç: GroupNorm(8, 120)

--##--##--##--##--

* ch = 122 (çift ama 8’e bölünmez)

* g=32 → 122%32≠0 → g=16 → 122%16≠0 → g=8 → 122%8≠0 → g=4 → 122%4≠0 → g=2

* 122%2=0, if çalışmaz

* Sonuç: GroupNorm(2, 122)

--##--##--##--##--

- ch = 121 (tek sayı)

* g=32 → 16 → 8 → 4 → 2 (while biter)

* 121%2≠0 → if devreye girer → tek olduğu için g=1

- Sonuç: GroupNorm(1, 121)



--------
---

# Shared Bottleneck Nedir? (Bu Kısımda Ne Yapılıyor?)

```python 
        self.conv1a = nn.Conv2d(in_channels, mid, 1, bias=False)
        self.norm1a = make_norm(norm, mid)
        self.conv1b = nn.Conv2d(mid, mid, 1, bias=False)
        self.norm1b = make_norm(norm, mid)
```



Aşağıdaki yapı, **Coordinate Attention içindeki ortak (shared) bottleneck** kısmıdır:

- Bu katmanlar **attention maskesi üretilmeden önce**
- H (yükseklik) ve W (genişlik) bilgisini **birlikte işleyen**
- Maskenin neye bakacağını belirleyen **en kritik karar bloğudur**



## Klasik Coordinate Attention’da Bottleneck Mantığı

Standart CA tasarımında genellikle şu vardır:

- Birleştirilmiş H+W bilgisi
- **Tek bir 1×1 convolution**
- Aktivasyon
- Sonra H ve W için ayrı head’ler

Bu yapı:
- Hafiftir
- Ama temsil gücü (capacity) sınırlıdır

Yani:
> “Maskeyi üretmeden önce özellikleri tek bir doğrusal dönüşümden geçiririm.”


## Bu Kodda Yapılan Şey: Bottleneck’i Derinleştirmek

Bu yapıda bottleneck **tek katman değil**, iki katmanlıdır:

- İlk katman: kanalı düşürür (`C → mid`)
- İkinci katman: `mid` uzayında tekrar işler (`mid → mid`)
- Her iki katmandan sonra:
  - Normalization
  - Nonlinearity (HSwish / ReLU)

Bu tasarım şunu sağlar:

> Attention maskesi üretilmeden önce, daha zengin ve daha ayrıştırıcı bir ortak temsil öğrenilir.


## Neden “Shared” Bottleneck Deniyor?

- H ve W yolları **ayrılmadan önce**
- Aynı dönüşümlerden geçer
- Yani:
  - Yükseklik bilgisi ve genişlik bilgisi
  - Aynı “karar mekanizması” tarafından işlenir

Bu, Coordinate Attention’ın temel fikrine sadıktır:
> “Koordinatlar ayrı, ama karar ortak.”


## Bu Neden Daha Güçlü?

### 1) Daha Fazla Nonlinearity
- Tek katman → tek karar yüzeyi
- İki katman → daha karmaşık ilişkiler öğrenilebilir

Bu özellikle şurada işe yarar:
- Mean + max pooling
- Depthwise + dilated conv
gibi zengin ama karmaşık sinyallerden anlamlı maske üretirken.



### 2) Mid Uzayında Ek İşleme
- `mid` kanalı bir “ara temsil”dir
- Bu uzayda ekstra dönüşüm yapmak:
  - Gürültüyü ayıklamaya
  - H/W bilgisini daha anlamlı sıkıştırmaya
yardımcı olur


### 3) Head’ler İçin Daha Temiz Girdi
Sonraki adımda:
- H ve W tekrar ayrılır
- Ayrı head’lerle kanal sayısına geri projekte edilir

Bu bottleneck ne kadar iyi olursa:
- `a_h` ve `a_w` maskeleri o kadar anlamlı olur

## Kısaca Özet

Bu blok:

- Convolution’ları “daha iyi” yapmak için değil
- **Attention maskesini üretmeden önceki ortak karar mekanizmasını güçlendirmek** için var



---
---

# Bu Katmanlar Ne İşe Yarıyor? (Depthwise “Yönlü” Konvolüsyon)

```python
        self.dw_h_d1 = nn.Conv2d(in_channels, in_channels, (3, 1), padding=(1, 0), groups=in_channels, bias=False)
        self.dw_w_d1 = nn.Conv2d(in_channels, in_channels, (1, 3), padding=(0, 1), groups=in_channels, bias=False)
```


Bu iki katman, Coordinate Attention’ın **H (yükseklik)** ve **W (genişlik)** yönlerinden çıkardığı özet sinyali daha “anlamlı” hale getirmek için var.

Buradaki temel fikir şu:

> “Pooling ile elde ettiğim 1D (tek eksenli) sinyali, küçük bir konvolüsyonla işleyip yerel bağlam (local context) ekleyeyim.”



## Neden `(3,1)` ve `(1,3)`?

Bu çekirdekler **yönlü (anisotropic) filtrelerdir**:

### `(3,1)` → Dikey yönde (H boyunca) bakar
- Yükseklik ekseninde komşu satırları görür
- Genişlik ekseninde ise tek kolonda kaldığı için yatay karışma yapmaz

Bu yüzden:
- **H yolundaki sinyali** “dikey komşuluk” bilgisiyle zenginleştirir



### `(1,3)` → Yatay yönde (W boyunca) bakar
- Genişlik ekseninde komşu sütunları görür
- Yükseklik ekseninde tek satırda kaldığı için dikey karışma yapmaz

Bu yüzden:
- **W yolundaki sinyali** “yatay komşuluk” bilgisiyle zenginleştirir



## Neden “Depthwise” (groups = in_channels)?

Bu katmanlar **depthwise convolution** olarak çalışır:

- Her kanal **kendi başına** filtrelenir
- Kanallar birbirine karışmaz

Bu şu anlama gelir:

### Artı tarafı
- Çok ucuz (parametre ve FLOP düşük)
- Kanal başına “ince ayar” yapar
- Attention tarafında gürültü eklemeden sinyali yumuşatır / şekillendirir

### Eksi tarafı
- Kanal karışımı yoktur  
  (Bu eksik daha sonra genelde 1×1 “merge” ile kapatılır.)


## Neden Padding Var?

- `(3,1)` için padding `(1,0)`:
  - H boyutu korunur (3’lük filtre H’de küçültmesin diye)
- `(1,3)` için padding `(0,1)`:
  - W boyutu korunur

Amaç:
> Bu işlemlerden sonra H/W özetlerinin boyutu değişmesin, aynı uzunlukta kalsın.



## Bu Tasarımın Coordinate Attention’a Katkısı

Klasik CA’da H ve W özetleri çoğu zaman sadece pooling’den gelir.  
Pooling tek başına bazen “fazla kaba” kalır.

Burada yapılan geliştirme:

- Pooling ile gelen **1D özet sinyal**
- küçük, yönlü, depthwise filtreyle işlenir
- böylece “yakın komşuluk ilişkileri” maskeye yansır

Özetle:

> “Maskeyi üretmeden önce H ve W sinyalini kanal bazında, düşük maliyetle, yönlü olarak zenginleştiriyoruz.”


---
---

# Burada Ne Yapıyoruz?

```python
        d = int(dilation)
        self.dw_h_d2 = nn.Conv2d(in_channels, in_channels, (3, 1), padding=(d, 0), dilation=(d, 1), groups=in_channels, bias=False)
        self.dw_w_d2 = nn.Conv2d(in_channels, in_channels, (1, 3), padding=(0, d), dilation=(1, d), groups=in_channels, bias=False)
```

- Zaten `(3,1)` ve `(1,3)` ile **yönlü depthwise** filtreler vardı (yakın komşuluk).
- Şimdi bunların bir de **dilation’lı** versiyonunu ekliyoruz:
  - H yönü için: `(3,1)` ama satırlar arasında “atlayarak” bakıyor
  - W yönü için: `(1,3)` ama sütunlar arasında “atlayarak” bakıyor

Bu yüzden toplamda 4 conv var:
- H: normal depthwise + dilated depthwise
- W: normal depthwise + dilated depthwise

Ama amaç “kanalı genişletmek” değil.
Amaç: **receptive field’i büyütmek** (yani daha geniş bağlam görmek).


## “3×1’i 5×1 mi yaptık?” ifadesini inceleyelim

Kısmen evet, ama doğru ifade şu:

**Dilated conv, kernel boyutunu büyütmez; kernel’in örnekleme aralığını büyütür.
Sonuç olarak “etkin kernel boyutu” büyür.**

Etkin kernel boyutu formülü

> Etkin boyut = (k - 1) * dilation + 1

Burada k=3:

* d=1 → etkin boyut: (3-1)*1 + 1 = 3 → 3

* d=2 → etkin boyut: (3-1)*2 + 1 = 5 → 5

* d=3 → etkin boyut: (3-1)*3 + 1 = 7 → 7

Yani dilation=2 iken:

* (3,1) etkin olarak (5,1) gibi davranır

* (1,3) etkin olarak (1,5) gibi davranır

Ama önemli nüans:

* Gerçek kernel hâlâ 3 elemanlıdır

* Sadece aralarında boşluk (atlama) vardır

## Neden hem normal hem dilated var?

Çünkü ikisi farklı tür bilgi toplar:

* Normal depthwise: ince detay, yakın komşuluk (local)

* Dilated depthwise: daha geniş bağlam, uzun menzil (context)

Bu ikisini toplamak:

**“Hem yakın hem uzak sinyali aynı anda yakala” demek.**

Coordinate Attention’da bu özellikle mantıklı çünkü:

* H ve W yönünde ürettiğin sinyal aslında “1D özet”

* Bu 1D özetin hem kısa hem uzun desenleri olabilir (ör. uzun çizgiler, şeritler, geniş objeler)


### Sonuç

* Evet: depthwise’ın dilated versiyonunu ekledik.

* Evet: toplam 4 conv oldu (H ve W için iki ölçek).

**Ama “kanalı genişletmek” değil; receptive field’i genişletmek.**

* dilation=2 ise (3,1) efektif olarak (5,1) gibi, (1,3) efektif olarak (1,5) gibi davranır.

---
---

```python
        self.merge_h = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        self.merge_w = nn.Conv2d(in_channels, in_channels, 1, bias=True)

        self.head_h = nn.Conv2d(mid, in_channels, 1, bias=True)
        self.head_w = nn.Conv2d(mid, in_channels, 1, bias=True)
```


# `merge_h / merge_w` Ne Yapıyor? 

Bu iki katman, H ve W yollarında **iki farklı ölçekte üretilen sinyali** (normal depthwise + dilated depthwise) birleştirdikten sonra devreye giriyor.

### Neyi “merge” ediyoruz?
H yolunda ve W yolunda şu fikir var:

- Aynı yön için iki farklı konvolüsyon çıktısı var:
  - **yakın bağlam** (normal depthwise)
  - **uzak bağlam** (dilated depthwise)

Bu iki çıktı önce toplanıyor (multi-scale fusion).  
Ama bu çıktılar **depthwise** olduğu için:

> Kanallar kendi içinde işlenmiş olsa da, kanallar arası etkileşim yoktur.

### 1×1 “merge” neden gerekli?
`merge_h` ve `merge_w` (1×1 conv) şu işi yapar:

- **kanallar arası karışım (channel mixing) sağlar**
- toplanan multi-scale sinyali “kanal boyutunda” yeniden düzenler
- gerekiyorsa küçük bir yeniden ölçekleme / kaydırma (bias) yapar

Yani doğru özet cümle:

> “Depthwise + dilated depthwise ile elde edilen H/W sinyalini birleştiriyoruz; sonra 1×1 ile kanallar arası etkileşim ekleyip sinyali ‘merge ediyoruz’.”

Not: Burada “2 kanaldan gelen” demek yerine, daha doğru ifade:
- “aynı kanal sayısında iki farklı dal/ölçekten gelen sinyali” merge ediyoruz.



## `head_h / head_w` Ne Yapıyor?

Bu iki katman Coordinate Attention’ın klasik mantığına geri dönüyor:

- Shared bottleneck (mid kanal) üzerinden işlenmiş temsil var.
- Bu temsil ikiye ayrılıyor:
  - H için olan parça
  - W için olan parça

Ama o parçaların kanal sayısı `mid`.  
Bizim nihai hedefimiz ise:

> H ve W için **C kanallı attention maskesi** üretmek.

### Head’ler bunun için var:
- `head_h`: `mid → in_channels (C)`
- `head_w`: `mid → in_channels (C)`

### 1×1 neden burada doğru seçim?
Çünkü head’lerin amacı:
- uzamsal boyutu büyütmek/küçültmek değil
- **kanal projeksiyonu yapmak** (mid uzayından tekrar C uzayına çıkmak)

Yani 1×1 burada şu anlama geliyor:

> “Her uzamsal konum için, kanallar üzerinde lineer bir dönüşüm yapıyorum.”

Bu sayede:
- H maskesi: `(B, C, H, 1)`
- W maskesi: `(B, C, 1, W)`
şeklinde çıkabilir.


## “Uzamsal boyuta getirdik” kısmını düzeltelim (önemli nüans)

Senin cümlende şöyle bir ifade var:
> “Head tarafında mid channels’a son çıktıyı kernel_size=1 ile yine uzamsal boyuta getirdik.”

Burada küçük ama önemli bir düzeltme var:

- `kernel_size=1` uzamsal boyutu “geri getirmez”.
- Uzamsal boyut zaten korunur (H veya W uzunluğu zaten var).
- 1×1’in yaptığı şey **kanal boyutunu dönüştürmek**tir.

Daha doğru cümle:

> “Head katmanları 1×1 conv ile, uzamsal boyutu sabit tutup kanal sayısını `mid → C` projekte eder; böylece H ve W için C-kanallı maskeler üretilir.”


## Kısa Özet

- **merge_h / merge_w**: Multi-scale (normal + dilated) depthwise çıktıyı birleştirip, 1×1 ile **kanal karışımı** ekler.
- **head_h / head_w**: Bottleneck’ten gelen `mid` temsili 1×1 ile **C kanala projekte eder**, H ve W attention maskelerini üretir.


---
---

# Bu Kısım Ne Yapıyor? (`alpha` için güvenli ve öğrenilebilir başlangıç)

```python
        eps = 1e-6
        a0 = float(init_alpha)
        a0 = min(max(a0, eps), 1.0 - eps)
        t = torch.tensor(a0)
        raw0 = torch.log(t / (1.0 - t))
```


Bu blok **çok teknik ama çok önemli** bir işi yapıyor:
> `alpha` değerini **öğrenilebilir** yapmak için, onu **logit uzayında güvenli şekilde başlatıyor**.

Adım adım, yavaş yavaş gidelim.


## 1️⃣ `eps = 1e-6` — Neden var?

```python
eps = 1e-6
```
Bu satırın tek amacı var:

* Sayısal patlamayı önlemek.

* Birazdan log(t / (1 - t)) hesaplanacak.

Eğer:

* t = 0 olursa → log(0) ❌

* t = 1 olursa → log(∞) ❌

İşte eps, bu uç durumları engellemek için.

## 2️⃣ a0 = float(init_alpha)
```python
a0 = float(init_alpha)
```


* Kullanıcıdan gelen init_alpha (ör. 0.7)

* Python float’a çevriliyor

* Henüz öğrenilebilir değil, sadece başlangıç değeri

Bu şu demek:

**“Attention başta ne kadar güçlü olsun istiyoruz?”**

## 3️⃣ a0 = min(max(a0, eps), 1.0 - eps)
```python
a0 = min(max(a0, eps), 1.0 - eps)
```


Bu satır clamp yapıyor.

Yani:

* a0 en az eps

* a0 en fazla 1 - eps

Başka bir deyişle:

**eps < a0 < 1 - eps**


Sebep:

* Birazdan log(t / (1 - t)) yapılacak

* Bu formül 0 ve 1’de tanımsız

Bu satır:

**“Ne olursa olsun sayısal olarak güvenli aralıkta kal.”**
## 4️⃣ t = torch.tensor(a0)
```python
t = torch.tensor(a0)
```


* Python float → PyTorch tensor

Çünkü birazdan:

* gradient

* parameter

* buffer
g
ibi şeylerle çalışacağız

Henüz:

* Parameter değil

* Sadece tensor

## 5️⃣ Asıl kritik satır: raw0 = log(t / (1 - t))
```python
raw0 = torch.log(t / (1.0 - t))
```


Burada yapılan şey şu:

* Sigmoid’in tersini (inverse / logit) almak

Hatırla:

Sigmoid fonksiyonu:
```bash
sigmoid(x) = 1 / (1 + exp(-x))
```


Bunun tersi (logit):
```bash
logit(p) = log(p / (1 - p))
```


Yani:

Eğer raw0’ı sigmoid’den geçirirsen:
```bash
sigmoid(raw0) = t = a0
```


🎯 Tam olarak istediğimiz şey bu.

## 6️⃣ Neden bu dolambaçlı yol?

Çünkü biz şunu istiyoruz:

* alpha 0 ile 1 arasında kalsın

* Ama öğrenilebilir (unconstrained) olsun

Bunun çözümü:

* Asıl parametreyi logit uzayında tut

* İleri geçişte sigmoid ile [0,1] aralığına map et

Yani:

* Öğrenilen şey: alpha_raw (−∞, +∞)

* Kullanılan şey: alpha = sigmoid(alpha_raw) ∈ (0,1)

Bu industry standard bir tekniktir.

---
---

# Bu Blok Ne Yapıyor? (Opsiyonel Spatial Gate)
```python 

        self.use_spatial_gate = bool(use_spatial_gate)
        self.spatial_gate_beta = float(spatial_gate_beta)
        if self.use_spatial_gate:
            self.spatial_dw = nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels, bias=False)
            self.spatial_pw = nn.Conv2d(in_channels, in_channels, 1, bias=True)
```



Bu kod, `use_spatial_gate=True` ise modele ek bir mekanizma ekler:

- Çıktı feature map üzerinde **konum bazlı (spatial)** bir maske üretir
- Bu maskeyi `sigmoid` ile 0–1 aralığına sıkıştırır
- Sonra bu maskeyi “yumuşatıp” (beta ile) feature map’e çarpar

Ama şunu unutma:
> Bu gate, CA’den bağımsız ikinci bir bastırma katmanı gibi çalışır.

## 1) `self.use_spatial_gate = bool(use_spatial_gate)`
Bu bir anahtar:

- `False` → bu gate tamamen devre dışı (hiç ek parametre yok)
- `True`  → aşağıdaki conv’lar eklenir ve forward’da uygulanır


## 2) `self.spatial_gate_beta = float(spatial_gate_beta)`
Bu, gate’in “şiddetini” kontrol eden katsayıdır.

Tipik mantık şu:
- gate sigmoid’den çıkar → (0,1)
- sonra `1 + beta*(gate - 1)` gibi bir yumuşatma ile 1’e yaklaştırılır  
  (yani gate çok agresif olmasın)

`beta` küçükse gate etkisi az, büyükse etkisi artar.


## 3) `self.spatial_dw` (3×3 depthwise conv)
Bu katman:

- **Spatial (H,W)** bağlamı görür (3×3 komşuluk)
- **Depthwise** olduğu için her kanalı ayrı işler (kanallar karışmaz)
- Amaç: konumsal olarak “nereler önemli / nereler değil” sinyali çıkarmak

Kısaca:
> “Her kanal için, 3×3 komşuluktan bir spatial ipucu çıkar.”


## 4) `self.spatial_pw` (1×1 pointwise conv)
Depthwise conv kanalları karıştırmadığı için 1×1 burada devreye girer:

- Kanal bazında yeniden ölçekleme / karıştırma sağlar
- “Gate” üretimini daha esnek yapar

Kısaca:
> “Depthwise’ın ürettiği sinyali kanal uzayında düzenleyip gerçek gate’e dönüştür.”


## Büyük Resim: Neden var?

Coordinate Attention zaten:
- H yönü maskesi (`a_h`)
- W yönü maskesi (`a_w`)
üretip bunları birleştiriyor.

Spatial gate ise şunu ekler:
- Doğrudan `(H,W)` üzerinde ek bir kapı

Yani:
- CA: koordinat temelli (ayrı H ve W)
- Spatial gate: klasik “spatial attention” gibi (tam 2D)


## Risk (Önemli)
Bu gate açılırsa:
- Aynı feature map ikinci kez bastırılabilir
- Bazı durumlarda fayda verir
- Bazı durumlarda over-suppression yapar (özellikle small batch / detection)

O yüzden default `False` mantıklı.


---
---
## Bu İki Satır Ne Yapıyor?
```python
        x_h = 0.5 * (x.mean(dim=3, keepdim=True) + x.amax(dim=3, keepdim=True))    # (B,C,H,1)
        x_w_1w = 0.5 * (x.mean(dim=2, keepdim=True) + x.amax(dim=2, keepdim=True)) # (B,C,1,W)
```



Elimizde giriş feature map var:

- `x` şekli: **(B, C, H, W)**  
  (Batch, Channel, Height, Width)

Burada amaç:
> Her kanal için, uzamsal bilgiyi iki farklı “şerit” özete çevirmek:
- **H yönü özeti:** (B, C, H, 1)
- **W yönü özeti:** (B, C, 1, W)

Bu, Coordinate Attention’ın “koordinat bilgisi kaybolmasın” diye yaptığı ana numara.



# 1) `x_h`: W eksenini sıkıştır, H’yi koru

`dim=3` demek: **W ekseni** (genişlik).

- `x.mean(dim=3, keepdim=True)`:
  - Her (B,C,H) için W boyunca **ortalama** alır.
  - Sonuç: **(B, C, H, 1)**

- `x.amax(dim=3, keepdim=True)`:
  - Her (B,C,H) için W boyunca **maksimum** alır.
  - Sonuç: **(B, C, H, 1)**

Sonra ikisini toplayıp 0.5 ile ortalamasını alıyorsun:

> **W boyunca “mean + max” karışımı ile bir H-profili çıkarıyorsun.**

Yani `x_h` şunu temsil eder:
> “Bu kanalda, her yükseklik satırında aktivasyon ne durumda?”  
(W’deki tüm piksellerin özetlenmiş hali)


## 2) `x_w_1w`: H eksenini sıkıştır, W’yi koru

`dim=2` demek: **H ekseni** (yükseklik).

- `x.mean(dim=2, keepdim=True)`:
  - H boyunca ortalama alır
  - Sonuç: **(B, C, 1, W)**

- `x.amax(dim=2, keepdim=True)`:
  - H boyunca maksimum alır
  - Sonuç: **(B, C, 1, W)**

Sonra yine 0.5 ile karıştırıyorsun:

> **H boyunca “mean + max” karışımı ile bir W-profili çıkarıyorsun.**

Yani `x_w_1w` şunu temsil eder:
> “Bu kanalda, her genişlik sütununda aktivasyon ne durumda?”  
(H’deki tüm piksellerin özetlenmiş hali)



## Neden Hem Mean Hem Max?

Klasik CA çoğu zaman sadece **mean pooling** kullanır.
Senin yaptığın geliştirme:

- **Mean**: genel dağılımı, “ortalama enerjiyi” yakalar  
- **Max**: en baskın/salient aktivasyonu yakalar (en güçlü sinyal nerede?)

Birlikte kullanınca:
> Hem “genel trend” hem “en dikkat çekici bölge” aynı anda maskeye sinyal verir.

Bu genelde attention’ı daha “keskin” ve daha bilgilendirici yapar.



## `keepdim=True` neden önemli?

Çünkü boyutu koruyup:
- `H` profilini **(B,C,H,1)** olarak,
- `W` profilini **(B,C,1,W)** olarak
tutmak istiyoruz.

Bu şekiller, biraz sonra concat/permute ile CA’nin standart akışına girecek.


## Tek cümlelik özet

> Bu iki satır, feature map’i kanal bazında iki adet 1D koordinat özetine çevirir; biri satır (H) profili, biri sütun (W) profili; üstelik mean+max ile daha güçlü bir özet üretir.


----
----

# H ve W Multi-Scale Bloku (Local + Dilated + 1×1 Merge)

```python 
        H_multi_scale = self.h_channel_mixer(self.h_local_dw(H_profile) + self.h_dilated_dw(H_profile))  

        W_multi_scale = self.w_local_dw(W_profile) + self.w_dilated_dw(W_profile)                        
        W_multi_scale = self.w_channel_mixer(W_multi_scale)                                               
        W_multi_scale = W_multi_scale.permute(0, 1, 3, 2) 
```


Bu kısımda yaptığımız işin özü şudur:

> H ve W için oluşturduğumuz 1D profilleri (şeritleri) iki farklı ölçekte (local + long-range) işleyip,
> sonra 1×1 ile kanal karıştırması yaparak “tek bir güçlü temsil” haline getiriyoruz.



## 1) `H_multi_scale` (Height profili için)

Elimizde şu var:
- **H_profile:** `(B, C, H, 1)`  
  (W ekseni sıkıştırıldı, H korunuyor)

Bu profile iki ayrı depthwise konvolüsyon uygulanıyor:

### a) Local (yakın komşuluk)
- H boyunca 3 elemanlık pencereyle bakar.
- “Kısa menzil / lokal detay” yakalar.

### b) Dilated (uzak bağlam)
- Yine 3 elemanlı pencere ama aralıklı örnekleme (dilation) ile.
- “Uzun menzil / geniş bağlam” yakalar.

Bu iki çıktı:

- Aynı şekildedir `(B, C, H, 1)`
- Aynı kanallara aittir (kanal sayısı değişmez)
- Bu yüzden toplanabilir.

> Buradaki toplama “kanal toplama” değil, **iki ölçeğin (multi-scale) birleştirilmesidir**.

### c) 1×1 “channel mixer” (merge)
Depthwise conv kanalları karıştırmadığı için, ardından 1×1 gelir:

- Kanallar arasında etkileşim sağlar (channel mixing)
- Multi-scale birleşimini kanal uzayında yeniden düzenler
- Her kanalın katkısını daha esnek hale getirir

Sonuç:
- **H_multi_scale:** `(B, C, H, 1)`



## 2) `W_multi_scale` (Width profili için)

Elimizde:
- **W_profile:** `(B, C, 1, W)`  
  (H ekseni sıkıştırıldı, W korunuyor)

### a) Local depthwise
- W boyunca lokal komşuluğu yakalar.

### b) Dilated depthwise
- W boyunca daha geniş bağlam yakalar (dilation ile).

İkisi yine aynı şekildedir `(B, C, 1, W)` ve toplanır:

> Bu da yine “kanal değil, ölçek birleştirme”dir.

### c) 1×1 “channel mixer” (merge)
Toplanan sinyal 1×1’den geçirilir:

- Kanal karışımı eklenir
- W yönündeki temsil daha ifade gücü kazanır

Sonuç hâlâ:
- `(B, C, 1, W)`

### d) `permute` (format uyumu için)
Son adımda şekli şu hale çeviriyoruz:
- `(B, C, 1, W)` → `(B, C, W, 1)`

Bunun sebebi:
> Biraz sonra H ve W’yi `dim=2` ekseninde `cat` edeceğiz.
> H tarafı zaten `(B, C, H, 1)` formatında.
> W tarafını da `(B, C, W, 1)` yaparsak,
> ikisini aynı eksende yan yana koyup `(B, C, H+W, 1)` oluşturabiliriz.



## Tek Cümlelik Özet

- H ve W profillerini **local + dilated depthwise** ile iki ölçekte işleriz,
- çıktıları **toplayıp** multi-scale birleştiririz,
- 1×1 ile **kanal mixing** ekleriz,
- W tarafını `permute` ile concat formatına hazırlarız.
---
---

# H + W Concat ve Shared Bottleneck Bloğu 

Bu bölümde artık **H ve W eksenlerinden gelen bilgiler ayrı ayrı değil**,  
**ortak bir karar mekanizması** içinde birlikte işlenir.

```python
        hw_concat = torch.cat([H_multi_scale, W_multi_scale], dim=2)                                       

        shared_mid = self.act(self.shared_bottleneck_norm(self.shared_bottleneck_proj(hw_concat)))       
        shared_mid = self.act(self.shared_bottleneck_refine_norm(self.shared_bottleneck_refine(shared_mid)))  

```


### 1️⃣ H ve W Şeritlerinin Birleştirilmesi (Concat)

Bu aşamaya gelmeden önce elimizde şunlar vardı:

- **H tarafı**:  
  Yükseklik boyunca anlam taşıyan, multi-scale işlenmiş bir şerit  
  → şekil olarak `(B, C, H, 1)`

- **W tarafı**:  
  Genişlik boyunca anlam taşıyan, multi-scale işlenmiş bir şerit  
  → şekil olarak `(B, C, W, 1)`

Bu iki şerit **toplanmaz**.  
Çünkü toplarsak bilgi üst üste biner ve karışır.

Bunun yerine:
> H şeridi ile W şeridi **yan yana eklenir**.

Sonuç:
- Kanal sayısı değişmez
- Uzunluk artık **H + W** olur
- Elimizde **tek bir uzun şerit** vardır

Bu şerit şu an şunu temsil eder:
> “Bu tensorun ilk kısmı H bilgisi, devamı W bilgisi.”


### 2️⃣ Neden Shared (Ortak) Bottleneck?

Buradaki temel fikir şudur:

> H ve W ayrı ayrı karar vermesin.  
> Önce **aynı masa etrafında otursunlar**,  
> sonra tekrar ayrılıp kendi maskelerini üretsinler.

Bu yüzden bu birleşik şerit **shared bottleneck** içine sokulur.



### 3️⃣ Bottleneck – 1. Aşama (C → mid)

İlk bottleneck adımında:

- Kanal boyutu **C’den mid’e düşürülür**
- Uzamsal uzunluk (**H+W**) korunur

Amaç:
- Hesap maliyetini düşürmek
- Gereksiz kanal bilgisini elemek
- Attention kararını daha sade bir uzayda vermek

Bu aşamadan sonra temsil:
- Daha kompakt
- Daha stabil
- Karar vermeye daha uygun hâle gelir



### 4️⃣ Bottleneck – 2. Aşama (mid → mid)

İkinci aşamada:

- Kanal boyutu **mid olarak kalır**
- Temsil bir kez daha işlenir (refine edilir)

Bu adımın amacı:
> “Tek katman yetmeyebilir;  
> ortak karar uzayında biraz daha düşünelim.”

Yani bu, bottleneck’i **derinleştirme** adımıdır.



### 5️⃣ Bu Bloğun Çıktısı Ne Anlama Geliyor?

Bu bloğun sonunda elimizde şu vardır:

- H ve W bilgisi **tek bir şerit hâlinde**
- Ama artık ham değil
- Ortak bir bottleneck’ten geçmiş
- Kanal boyutu küçültülmüş
- “Karar vermeye hazır” bir temsil

Bir sonraki adımda:
- Bu uzun şerit tekrar **H ve W olarak ikiye ayrılacak**
- Her biri kendi attention maskesini üretmek için kullanılacak


### 🔑 Tek Cümlelik Özet

H ve W’den gelen multi-scale şeritleri yan yana ekleyip,  
kanal boyutunu düşüren shared bottleneck’ten geçirerek  
iki eksen için **ortak bir attention karar temsili** üretiyoruz.


---
---
# Split + Permute + Head (H ve W Maskelerini Üretme Aşaması)

Bu blokta artık “ortak karar şeridi” olan `shared_mid` içinden  
**H ve W parçalarını geri ayırıp**, her biri için **attention maskesi** üretiyoruz.

```python 
        mid_H, mid_W = torch.split(shared_mid, [H, W], dim=2)                                             
        mid_W = mid_W.permute(0, 1, 3, 2)                                                                   

        attn_H = F.hardsigmoid(self.h_attention_head(mid_H), inplace=False)                                 
        attn_W = F.hardsigmoid(self.w_attention_head(mid_W), inplace=False) 
```


### 1️⃣ `torch.split(shared_mid, [H, W], dim=2)` ne yapıyor?

`shared_mid` şu formdaydı:

- **(B, mid, H+W, 1)**

Burada `H+W` uzunluğu **dim=2** ekseninde duruyor.

Önceden `cat` yaparken:
- önce H şeridi
- sonra W şeridi
şeklinde yan yana koymuştuk.

Split burada şunu diyor:

> “dim=2 eksenini, önce **H uzunluğunda**, sonra **W uzunluğunda** iki parçaya ayır.”

Sonuçlar:

- **mid_H**: `(B, mid, H, 1)`  
  → H eksenine ait bottleneck temsili

- **mid_W**: `(B, mid, W, 1)`  
  → W eksenine ait bottleneck temsili

**Önemli:** split yüzdeyle bölmez; senin verdiğin kesin uzunluklara göre böler.



### 2️⃣ `mid_W = mid_W.permute(0, 1, 3, 2)` neden var?

Split sonrası `mid_W` şu şekilde:

- `(B, mid, W, 1)`

Ama W için maskeyi üretirken (head_w tarafında) standart format şu olmalı:

- `(B, mid, 1, W)`

Yani W uzunluğunu **son eksene** taşımamız gerekiyor.

`permute(0,1,3,2)` şu değişimi yapar:

- `(B, mid, W, 1)` → `(B, mid, 1, W)`

Bu tamamen **format uyumu** içindir; bilgi değişmiyor, sadece eksenler yer değiştiriyor.


### 3️⃣ `h_attention_head(mid_H)` ve `w_attention_head(mid_W)` ne yapıyor?

Şimdi iki parça da “maskeye dönüşme” aşamasına geldi.

Bu head katmanlarının görevi:

> Bottleneck’teki (mid kanallı) temsili,  
> tekrar **orijinal kanal sayısına (C)** projekte etmek.

Yani:
- **mid → C** dönüşümü

Bu yüzden head’ler 1×1 konv gibi davranır:
- Uzamsal boyut korunur
- Kanal boyutu değiştirilir



### 4️⃣ `F.hardsigmoid(..., inplace=False)` neden kullanılıyor?

Head çıktısı “maskeye benzer” bir şeye dönüştürülmek istenir:
- değerler kontrollü olsun
- aşırı büyük/küçük değerler taşmasın

`hardsigmoid` burada:
- maskeyi **0–1 aralığına** sıkıştırır (pratikte)
- sigmoid’e göre daha ucuz ve stabil bir aktivasyon olabilir

`inplace=False` seçimi:
- autograd ve ara tensörlerin güvenliği açısından daha az riskli
- inplace bazen gradient graph’ını bozabilir, bu tür attention akışlarında gereksiz risk


### 5️⃣ Bu bloğun çıktıları ne?

Sonuçta elde ettiğimiz iki maskedir:

- **attn_H**: `(B, C, H, 1)`  
  → “H ekseninde, her kanal için ölçek”

- **attn_W**: `(B, C, 1, W)`  
  → “W ekseninde, her kanal için ölçek”

Bir sonraki aşamada:
- bu iki maske alpha/beta ile yumuşatılacak
- çarpılıp `(B,C,H,W)` tam 2D ölçeğe dönüştürülecek
- ve giriş feature map’e uygulanacak



### 🔑 Tek Cümlelik Özet

`shared_mid` içindeki H+W şeridini tekrar H ve W olarak ayırıp,  
W’yi formata sokup, iki head ile `mid → C` projekte ederek  
H ve W için kanal bazlı attention maskeleri üretiyoruz.

---
---



# `alpha` ve `scale_H / scale_W` Bloğu (Kontrollü Attention Gücü)

Bu bölüm, üretilen `attn_H` ve `attn_W` maskelerini **doğrudan uygulamak yerine**
önce **“ne kadar güveneceğiz?”** sorusuna cevap veren bir karışım (mixing) uygular.

```python
        alpha_h = torch.sigmoid(self.alpha_h_raw)# Bu alpha değerleri                                                 
        alpha_w = torch.sigmoid(self.alpha_w_raw)                                                          

        scale_H = (1.0 - alpha_h) + alpha_h * attn_H  # Burda dierktr scale matematiğini kulanadık :: scale=(1−α)+α⋅attn                                                   
        scale_W = (1.0 - alpha_w) + alpha_w * attn_W                                                         

```



### 1️⃣ `alpha_h = sigmoid(alpha_h_raw)` ne demek?

`alpha_h_raw` / `alpha_w_raw` aslında **sınırsız** değer alabilen parametrelerdir:
- teorik aralık: `(-∞, +∞)`

Ama `alpha` değerinin anlamı şudur:
> “Attention’ı ne kadar açayım?”

Bu da doğal olarak **0 ile 1 arasında** olmalıdır.

Bu yüzden `sigmoid` ile:
- `alpha_h = sigmoid(alpha_h_raw)` → `(0, 1)`
- `alpha_w = sigmoid(alpha_w_raw)` → `(0, 1)`



> Not: `alpha_raw`’ın “logit uzayında” tutulması, öğrenmeyi daha rahat yapar;
> çünkü parametre serbestçe hareket eder, ama `alpha` yine güvenli aralıkta kalır.



### 2️⃣ `scale_H = (1-α) + α·attn_H` ne yapıyor?

Bu formül, matematikte çok bilinen bir şeydir:
> **Linear interpolation (lerp)**

Yani `scale_H`, iki uç arasında karışım yapar:
- **1.0** (hiç dokunma / identity)
- **attn_H** (tam attention)

Bunu `α` kontrol eder:

#### ✅ Eğer `α = 0` olursa
\[
scale_H = (1-0) + 0 \cdot attn_H = 1
\]
→ Attention tamamen devre dışı: **hiç ölçekleme yok**

#### ✅ Eğer `α = 1` olursa
\[
scale_H = (1-1) + 1 \cdot attn_H = attn_H
\]
→ Attention tam güç: **maskeyi olduğu gibi uygula**

#### ✅ Eğer `0 < α < 1` olursa
\[
scale_H = 1 \text{ ile } attn_H \text{ arasında bir değer}
\]
→ Attention “yumuşak” uygulanır, agresiflik düşer

Aynı mantık `scale_W` için de geçerlidir.



### 3️⃣ Neden böyle bir karışım gerekli?

Çünkü `attn_H` / `attn_W` doğrudan kullanılırsa:
- bazı durumlarda aşırı bastırma (over-suppression) olabilir
- eğitim başında maskeler yanlış kalibre olabilir
- küçük batch / gürültülü feature’larda stabilite düşebilir

Bu karışım sayesinde:
> Model isterse attention’ı **zamanla açar**,
> isterse **daha kapalı** tutar.

Yani `alpha`, **öğrenilebilir bir “güç düğmesi”** gibi çalışır.



### 🔑 Tek Cümlelik Özet

`alpha` ile maskeyi doğrudan uygulamak yerine, maskeyi **1 ile attn arasında karıştırarak**
attention’ın şiddetini öğrenilebilir ve stabil hale getiriyoruz.


---
----
# Final Ölçekleme (`beta`) + Uygulama + Opsiyonel Spatial Gate

Bu son blok, üretilen 2D ölçeği (scale) **daha güvenli hale getirip** girişe uygular;  
istersen de bunun üstüne **ek bir lokal 2D gate** koyar.

```python
        out = x * scale_hw # her kanalın her pikseli, kendi ölçeğiyle çarpılıyor.                                                                               

        if self.use_spatial_gate:# Bu blok, CA’den sonra çıktıya ek bir 2D lokal kapı uygular; bazen faydalı, bazen fazla bastırıp performansı bozabilir.Eğer kullanılacaksa dikkat edilmeli.
            gate = torch.sigmoid(self.spatial_gate_pw(self.spatial_gate_dw(out)))                            
            gate = 1.0 + self.spatial_gate_beta * (gate - 1.0)                                              
            out = out * gate                                                                                 

        return out
```


### 1️⃣ `scale_hw = 1.0 + beta * (scale_hw - 1.0)` ne yapıyor?

Bu satır, `scale_hw` değerlerini **1’e doğru çeker**.

- `scale_hw` aslında H ve W maskelerinden türetilen 2D ölçek haritasıdır.
- Bazı durumlarda bu ölçek fazla agresif olabilir (çok bastırma / fazla şişirme).

Bu formül şudur:

> “Scale’i olduğu gibi kullanma, etkisini `beta` kadar uygula.”

#### Beta’nın etkisi:
- **beta = 0**  
  → `scale_hw = 1`  
  → tamamen identity (hiç attention yok gibi)

- **beta = 1**  
  → `scale_hw` olduğu gibi kullanılır (tam etki)

- **0 < beta < 1**  
  → scale 1’e yaklaşır  
  → bastırma/şişirme yumuşar, stabilite artar

Bunu pratikte şöyle düşün:
> `beta` = attention’ın “genel sesini kısma düğmesi”.

### 2️⃣ `out = x * scale_hw` ne demek?

Bu, attention’ın uygulandığı yer:

- `x`: orijinal feature map `(B, C, H, W)`
- `scale_hw`: her kanal ve her piksel için ölçek `(B, C, H, W)`

Çarpım şu anlama gelir:

> “Her kanalın her konumu, kendi ölçeği kadar büyütülür veya küçültülür.”

Bu noktada attention artık **tamamen uygulanmış** olur.



### 3️⃣ Opsiyonel: `use_spatial_gate` bloğu ne yapar?

Bu kısım açılırsa (True):

> CA’den çıkmış `out` üzerinde **ek bir 2D lokal kapı (gate)** üretilir ve tekrar çarpılır.

Bu gate üretimi:

- `spatial_gate_dw`: 3×3 depthwise  
  → lokal 2D komşuluğu görür (H,W), kanal kanal işler
- `spatial_gate_pw`: 1×1  
  → kanal bazında karıştırma / ayarlama
- `sigmoid`: gate’i 0–1 aralığına sıkıştırır

Sonra gate yine `spatial_gate_beta` ile 1’e çekilerek yumuşatılır:

> “Gate çok agresif olmasın, etkisini kontrollü uygula.”

Son adım:
- `out = out * gate`  
  → CA’den geçen çıktı bir kez daha “lokal 2D kontrol” ile düzenlenmiş olur



### 4️⃣ Neden “riskli” diyorsun?

Çünkü iki kez ölçekleme yapmış oluyorsun:

1) CA ölçeği ile `x * scale_hw`
2) Spatial gate ile `out * gate`

Bu bazı senaryolarda:
- aşırı bastırma (over-suppression)
- zayıf gradient akışı
- detection + küçük batch’te dalgalanma
gibi sorunlar çıkarabilir.

Bu yüzden:
- Default kapalı olması mantıklı
- Açacaksan `spatial_gate_beta` genelde küçük tutulmalı



### 🔑 Tek Cümlelik Özet

Önce `beta` ile CA ölçeğini 1’e yaklaştırıp agresifliği düşürüyoruz,  
sonra feature map’e uyguluyoruz; opsiyonel spatial gate açıksa, çıktıya bir de lokal 2D kapı ekleyip ikinci bir kontrollü ölçekleme yapıyoruz.


